In [1]:
# kaggle website link: https://www.kaggle.com/c/bike-sharing-demand

In [32]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
train_data = pd.read_csv("./Data/train.csv")
test_data = pd.read_csv("./Data/test.csv")

In [4]:
train_data.shape, test_data.shape

((10886, 12), (6493, 9))

### Inspecting train and test data columns for irregularities

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    6493 non-null   object 
 1   season      6493 non-null   int64  
 2   holiday     6493 non-null   int64  
 3   workingday  6493 non-null   int64  
 4   weather     6493 non-null   int64  
 5   temp        6493 non-null   float64
 6   atemp       6493 non-null   float64
 7   humidity    6493 non-null   int64  
 8   windspeed   6493 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 456.7+ KB


### There are no nulls in the dataset so no manipulation needed for this step

In [7]:
train_data.isna().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

### There is no 'casual' and 'registered' columns in the test data. It seems adding those columns together will equal the count column. Will check to confirm. If that is the case, then columns will be dropped.

In [8]:
train_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [9]:
train_data['count_check'] = train_data['casual'] + train_data['registered']
train_data['count'].equals(train_data['count_check'])

True

### The casual and registered columns equal the count so will drop those columns and the count_check column

In [10]:
train_data = train_data.drop(columns=['casual', 'registered', 'count_check'])
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   count       10886 non-null  int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 850.6+ KB


### The train and test data were split by the days of the month (train data is from the 1st to the 19th and the test data is from the 20th to the end of the month.) The datetime column will be dropped after three new columns are created: 'month', 'hour',and 'day of the week' column.

In [11]:
# convert to column to datetime
train_data['datetime'] = pd.to_datetime(train_data['datetime'])

In [12]:
train_data['month'] = pd.DatetimeIndex(train_data['datetime']).month
train_data['hour'] = pd.DatetimeIndex(train_data['datetime']).hour
train_data['dayofweek'] = pd.DatetimeIndex(train_data['datetime']).dayofweek
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   count       10886 non-null  int64         
 10  month       10886 non-null  int64         
 11  hour        10886 non-null  int64         
 12  dayofweek   10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(9)
memory usage: 1.1 MB


In [13]:
train_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,month,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16,1,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32,1,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,13,1,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,1,1,4,5


In [14]:
# drop datetime column
train_data = train_data.drop(columns='datetime')

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   atemp       10886 non-null  float64
 6   humidity    10886 non-null  int64  
 7   windspeed   10886 non-null  float64
 8   count       10886 non-null  int64  
 9   month       10886 non-null  int64  
 10  hour        10886 non-null  int64  
 11  dayofweek   10886 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 1020.7 KB


#### Will One hot encode categorical columns and standardize cont colums and then recombine them with cont columns

In [16]:
#  'count' = number of total rentals
y = train_data['count']

cat_cols = ['season', 'holiday', 'workingday', 'weather', 'month', 'hour', 'dayofweek']

cont_cols = ['temp', 'atemp', 'humidity', 'windspeed']

In [17]:
cat_df = train_data[cat_cols].astype('str')

In [18]:
cont_df = train_data[cont_cols]

In [19]:
dummied_df = pd.get_dummies(cat_df, drop_first=True)


In [20]:
dummied_df.head()

,season_2,season_3,season_4,holiday_1,workingday_1,weather_2,weather_3,weather_4,month_10,month_11,...,hour_6,hour_7,hour_8,hour_9,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### Will standard scale the cont columns

In [26]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(cont_df)
scaled_df= pd.DataFrame(scaled_df, columns=cont_cols)
scaled_df

,temp,atemp,humidity,windspeed
0,-1.333661,-1.092737,0.993213,-1.567754
1,-1.438907,-1.182421,0.941249,-1.567754
2,-1.438907,-1.182421,0.941249,-1.567754
3,-1.333661,-1.092737,0.681430,-1.567754
4,-1.333661,-1.092737,0.681430,-1.567754
...,...,...,...,...
10881,-0.596935,-0.467310,-0.617666,1.617227
10882,-0.702182,-0.735182,-0.253919,0.269704
10883,-0.807428,-0.913959,-0.046064,0.269704
10884,-0.807428,-0.735182,-0.046064,-0.832442


#### Recombine cat and cont columns

In [27]:
processed_train_data = pd.concat([scaled_df, dummied_df], axis=1)

In [31]:
processed_train_data.shape

(10886, 52)

# Modelling

In [30]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(processed_train_data, y)
dummy_regr.predict(processed_train_data)
dummy_regr.score(processed_train_data, y)

0.0

In [34]:
linreg = LinearRegression()

linreg.fit(processed_train_data, y)
linreg.score(processed_train_data, y)

0.6378685340119912